In [1]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from RBM import RBM
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 100
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

2022-05-06 13:35:35.559013 model.py code start


In [2]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
dataset_db1, dataset_db2, dataset_db3 = mf.ecg_filtering(True)

train_dataset = list(mf.list_to_list(dataset_db1)) * 4
cross_dataset = list(mf.list_to_list(dataset_db2)) * 4
test_dataset = list(mf.list_to_list(dataset_db3))  * 4

X_train, X_test, y_train, y_test = train_test_split(
    (train_dataset + cross_dataset), 
    (test_dataset + cross_dataset),
    test_size=0.33,
    shuffle=True
)

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read file and indexing start...
[INFO]	final_db1 direcotry found.
......	...................i	Current_Index	From_Index
[IWIP]	final_db1 reading... 0 0 200
[IWIP]	final_db1 reading... 1 200 400
[IWIP]	final_db1 reading... 2 400 600
[IWIP]	final_db1 reading... 3 600 800
[IWIP]	final_db1 reading... 4 800 1000
[IWIP]	final_db1 reading... 5 1000 1200
[IWIP]	final_db1 reading... 6 1200 1400
[IWIP]	final_db1 reading... 7 1400 1600
[IWIP]	final_db1 reading... 8 1600 1800
[IWIP]	final_db1 reading... 9 1800 2000
[IWIP]	final_db1 reading... 10 2000 2200
[IWIP]	final_db1 reading... 11 2200 2400
[IWIP]	final_db1 reading... 12 2400 2600
[IWIP]	final_db1 reading... 13 2600 2800
[IWIP]	final_db1 reading... 14 2800 3000
[IWIP]	final_db1 reading... 15 3000 3200
[IWIP]	final_db1 reading... 16 3200 3400
[IWIP]	final_db1 reading... 17 3400 3600
[IWIP]	final_db1 reading... 18 

/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/scipy/signal/_signaltools.py:1531: UserWarning: kernel_size exceeds volume extent: the volume will be zero-padded.
  warnings.warn('kernel_size exceeds volume extent: the volume will be '


In [3]:
train_dataloader = DataLoader(X_train + y_train,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(X_test + y_test,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

print("X_train length : ", len(X_train))
print("X_test  length : ", len(X_test))
print("y_train length : ", len(y_train))
print("y_test  length : ", len(y_test))

train_data = torch.FloatTensor(X_train)
test_data = torch.FloatTensor(X_test)

X_train length :  9648
X_test  length :  4752
y_train length :  9648
y_test  length :  4752


In [4]:
print("[INFO] Model object added")

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

[INFO] Model object added


In [5]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for _, (data) in enumerate(train_dataloader):
        try:
            # tnesor float
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_second, v2, mt = rbm_second(sample_data)
        
        loss_second = rbm_second.free_energy(vog_second) - rbm_second.free_energy(v2)
        loss_.append(loss_second.data)
        
        second_train_op.zero_grad()
        loss_second.backward()
        second_train_op.step()

    output_from_second.append(v2.tolist())
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third bbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_third, v3, mt = rbm_third(sample_data)
        
        loss_third = rbm_third.free_energy(vog_third) - rbm_third.free_energy(v3)
        loss_.append(loss_third.data)
        
        third_train_op.zero_grad()
        loss_third.backward()
        third_train_op.step()

    output_from_third.append(v3.tolist())
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/3670610706.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1ST BBrbm_first Training loss for 0 epoch -0.15082305669784546	Estimate time : 0:00:00.000135
1ST BBrbm_first Training loss for 1 epoch -0.02662709727883339	Estimate time : 0:00:00.000135
1ST BBrbm_first Training loss for 2 epoch -0.04441321641206741	Estimate time : 0:00:00.000135
1ST BBrbm_first Training loss for 3 epoch -0.02389226108789444	Estimate time : 0:00:00.000136
1ST BBrbm_first Training loss for 4 epoch 0.002684774110093713	Estimate time : 0:00:00.000134
1ST BBrbm_first Training loss for 5 epoch -0.02136393077671528	Estimate time : 0:00:00.000136
1ST BBrbm_first Training loss for 6 epoch -0.06895525008440018	Estimate time : 0:00:00.000145
1ST BBrbm_first Training loss for 7 epoch -0.004774047061800957	Estimate time : 0:00:00.000135
1ST BBrbm_first Training loss for 8 epoch 0.009043700061738491	Estimate time : 0:00:00.000136
1ST BBrbm_first Training loss for 9 epoch 0.01466924324631691	Estimate time : 0:00:00.000134
1ST BBrbm_first Training loss for 10 epoch -0.01388026867061

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/3670610706.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST BBrbm_first Training loss for 0 epoch 0.005755618214607239	Estimate time : 
2ST BBrbm_first Training loss for 1 epoch 0.005657751578837633	Estimate time : 
2ST BBrbm_first Training loss for 2 epoch 0.005415000487118959	Estimate time : 
2ST BBrbm_first Training loss for 3 epoch 0.005282445345073938	Estimate time : 
2ST BBrbm_first Training loss for 4 epoch 0.005186568945646286	Estimate time : 
2ST BBrbm_first Training loss for 5 epoch 0.005121611058712006	Estimate time : 
2ST BBrbm_first Training loss for 6 epoch 0.005051665939390659	Estimate time : 
2ST BBrbm_first Training loss for 7 epoch 0.005166582763195038	Estimate time : 
2ST BBrbm_first Training loss for 8 epoch 0.005149821285158396	Estimate time : 
2ST BBrbm_first Training loss for 9 epoch 0.005184765439480543	Estimate time : 
2ST BBrbm_first Training loss for 10 epoch 0.0051756007596850395	Estimate time : 
2ST BBrbm_first Training loss for 11 epoch 0.005216912366449833	Estimate time : 
2ST BBrbm_first Training loss for 12 

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/3670610706.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST BBrbm_first Training loss for 0 epoch -0.0031922035850584507	Estimate time : 
3ST BBrbm_first Training loss for 1 epoch -0.003636738983914256	Estimate time : 
3ST BBrbm_first Training loss for 2 epoch -0.0038918291684240103	Estimate time : 
3ST BBrbm_first Training loss for 3 epoch -0.004353489261120558	Estimate time : 
3ST BBrbm_first Training loss for 4 epoch -0.004142201971262693	Estimate time : 
3ST BBrbm_first Training loss for 5 epoch -0.004268793389201164	Estimate time : 
3ST BBrbm_first Training loss for 6 epoch -0.004141347948461771	Estimate time : 
3ST BBrbm_first Training loss for 7 epoch -0.0047388020902872086	Estimate time : 
3ST BBrbm_first Training loss for 8 epoch -0.004833575803786516	Estimate time : 
3ST BBrbm_first Training loss for 9 epoch -0.004874990321695805	Estimate time : 
3ST BBrbm_first Training loss for 10 epoch -0.0050459629856050014	Estimate time : 
3ST BBrbm_first Training loss for 11 epoch -0.004523877985775471	Estimate time : 
3ST BBrbm_first Traini

In [6]:
    
print("BBRBM is done.")
print("GBRBM is start")

output_from_first = list()
output_from_second = list()
output_from_third = torch.tensor(output_from_third)

rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

# print(output_from_third.size(), output_from_third.dim(), "\n", output_from_third)
gaussian_std = torch.arange(1, 0, -0.1)

for epoch in range(EPOCH):
    '''First gbrbm'''
    for _, (data) in enumerate(output_from_third):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_first, gb_v1, mt = rbm_first(sample_data)
        
        gb_loss_first = rbm_first.free_energy(gb_vog_first) - rbm_first.free_energy(gb_v1)
        loss_.append(gb_loss_first.data)
        
        gb_first_train_op.zero_grad()
        gb_loss_first.backward()
        gb_first_train_op.step()

    output_from_first.append(gb_v1.tolist())
    print("1ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_first = torch.tensor(output_from_first)
for epoch in range(EPOCH):
    '''Second gbrbm'''
    for _, (data) in enumerate(output_from_first):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_second, gb_v2, mt = rbm_second(sample_data)
        
        gb_loss_second = rbm_second.free_energy(gb_vog_second) - rbm_second.free_energy(gb_v2)
        loss_.append(gb_loss_second.data)
        
        gb_second_train_op.zero_grad()
        gb_loss_second.backward()
        gb_second_train_op.step()

    output_from_second.append(gb_v2.tolist())
    print("2ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))

output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Third gbrbm'''
    for _, (data) in enumerate(output_from_second):
        try:
            data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        except RuntimeError:
            continue
        
        # CHANGED to GAUSSIAN
        sample_data = torch.normal(mean=data, std=gaussian_std)
        sample_data = torch.flatten(sample_data.clone())

        gb_vog_third, gb_v3, mt = rbm_third(sample_data)
        
        gb_loss_third = rbm_third.free_energy(gb_vog_third) - rbm_second.free_energy(gb_v3)
        loss_.append(gb_loss_third.data)
        
        gb_third_train_op.zero_grad()
        gb_loss_third.backward()
        gb_third_train_op.step()

    print("3ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))


BBRBM is done.
GBRBM is start


/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/2809123372.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


1ST GBrbm_first Training loss for 0 epoch -0.014309344813227654	Estimate time : 0:00:00.000154
1ST GBrbm_first Training loss for 1 epoch -0.01428990252315998	Estimate time : 0:00:00.000150
1ST GBrbm_first Training loss for 2 epoch -0.014273679815232754	Estimate time : 0:00:00.000166
1ST GBrbm_first Training loss for 3 epoch -0.01425878331065178	Estimate time : 0:00:00.000134
1ST GBrbm_first Training loss for 4 epoch -0.014239804819226265	Estimate time : 0:00:00.000135
1ST GBrbm_first Training loss for 5 epoch -0.01422201469540596	Estimate time : 0:00:00.000135
1ST GBrbm_first Training loss for 6 epoch -0.014203899540007114	Estimate time : 0:00:00.000139
1ST GBrbm_first Training loss for 7 epoch -0.014188534580171108	Estimate time : 0:00:00.000135
1ST GBrbm_first Training loss for 8 epoch -0.014172779396176338	Estimate time : 0:00:00.000145
1ST GBrbm_first Training loss for 9 epoch -0.014156419783830643	Estimate time : 0:00:00.000140
1ST GBrbm_first Training loss for 10 epoch -0.0141388

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/2809123372.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


2ST GBrbm_first Training loss for 0 epoch -0.012671638280153275	Estimate time : 0:00:00.000137
2ST GBrbm_first Training loss for 1 epoch -0.012661315500736237	Estimate time : 0:00:00.000163
2ST GBrbm_first Training loss for 2 epoch -0.012651071883738041	Estimate time : 0:00:00.000137
2ST GBrbm_first Training loss for 3 epoch -0.012640893459320068	Estimate time : 0:00:00.000144
2ST GBrbm_first Training loss for 4 epoch -0.012631632387638092	Estimate time : 0:00:00.000135
2ST GBrbm_first Training loss for 5 epoch -0.012621307745575905	Estimate time : 0:00:00.000134
2ST GBrbm_first Training loss for 6 epoch -0.012612070888280869	Estimate time : 0:00:00.000137
2ST GBrbm_first Training loss for 7 epoch -0.01260171178728342	Estimate time : 0:00:00.000140
2ST GBrbm_first Training loss for 8 epoch -0.012591542676091194	Estimate time : 0:00:00.000141
2ST GBrbm_first Training loss for 9 epoch -0.012581631541252136	Estimate time : 0:00:00.000142
2ST GBrbm_first Training loss for 10 epoch -0.01257

/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/2809123372.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)


3ST GBrbm_first Training loss for 0 epoch -0.01174195297062397	Estimate time : 0:00:00.000137
3ST GBrbm_first Training loss for 1 epoch -0.011760997585952282	Estimate time : 0:00:00.000135
3ST GBrbm_first Training loss for 2 epoch -0.01178120356053114	Estimate time : 0:00:00.000133
3ST GBrbm_first Training loss for 3 epoch -0.011800589971244335	Estimate time : 0:00:00.000134
3ST GBrbm_first Training loss for 4 epoch -0.011816178448498249	Estimate time : 0:00:00.000135
3ST GBrbm_first Training loss for 5 epoch -0.011833860538899899	Estimate time : 0:00:00.000135
3ST GBrbm_first Training loss for 6 epoch -0.011851143091917038	Estimate time : 0:00:00.000133
3ST GBrbm_first Training loss for 7 epoch -0.011869900859892368	Estimate time : 0:00:00.000140
3ST GBrbm_first Training loss for 8 epoch -0.011884655803442001	Estimate time : 0:00:00.000134
3ST GBrbm_first Training loss for 9 epoch -0.011900849640369415	Estimate time : 0:00:00.000135
3ST GBrbm_first Training loss for 10 epoch -0.011919

In [7]:
nprst = gb_v3.detach().numpy()
print(nprst)

rbm_first.get_weight()

[1. 1. 1. 1. 0. 1. 0. 0. 1. 1.]


Parameter containing:
tensor([[-0.0102,  0.0017,  0.0037, -0.0122, -0.0016, -0.0070,  0.0028,  0.0069,
          0.0276,  0.0120]], requires_grad=True)

In [8]:
test_loss = 0
train_loss = 0
train_cnt = 0
summary_c = 0

for _, test_data in enumerate(test_dataloader):
    try:
        test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
        train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)
    except RuntimeError:
        pass
    
    testing_data = torch.flatten(torch.bernoulli(test_data))
    training_data = torch.flatten(torch.bernoulli(train_data))
    
    vt, vt1, _ = rbm_first(testing_data)
    test_loss = rbm_first.free_energy(vt) - rbm_first.free_energy(vt1)    
    
    vs, vs1, _ = rbm_first(training_data)
    train_loss = rbm_first.free_energy(vs) - rbm_first.free_energy(vs1)

    
    test_loss += torch.mean(torch.abs(vt1[vt1 >= 0] - vt[vt1 >= 0]))
    # print(vt1[vt1 >= 0] - vt[vt1 >= 0])
    summary_c += 1

print('Test loss : ' + str(test_loss / summary_c))
print('Train - test : ' + str(train_loss - test_loss))


/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/200555237.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = torch.tensor(Variable(data.view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
/var/folders/_8/q_cwh5hn0s1dxsrzq2d040p80000gn/T/ipykernel_58328/200555237.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor((Variable(train_data[train_cnt:train_cnt + 10])).uniform_(0, 1), dtype=torch.float32)


Test loss : tensor(0.0009, grad_fn=<DivBackward0>)
Train - test : tensor(-0.8119, grad_fn=<SubBackward0>)


In [9]:
'''Test code'''
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

output_from_first = list()
output_from_second = list()
output_from_third = list()

test_loss = 0
epoch_cnt = 0

In [10]:
'''First BBRBM Guide Line'''

for _, data in enumerate(test_dataloader):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v1, vt1, _ = rbm_first(data)
    test_loss += rbm_first.free_energy(v1) - rbm_first.free_energy(vt1)
    epoch_cnt += 1
    output_from_first.append(vt1.tolist())
print('\tBBRBM_First_layer test loss : ', str(test_loss / epoch_cnt))

'''Second BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_first)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v2, vt2, _ = rbm_second(data)
    test_loss += rbm_second.free_energy(v2) - rbm_second.free_energy(vt2)
    epoch_cnt += 1
    output_from_second.append(vt2.tolist())
print('\tBBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

'''Third BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_second)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    data = torch.flatten(torch.bernoulli(test_data))
    
    v3, vt3, _ = rbm_third(data)
    test_loss += rbm_third.free_energy(v3) - rbm_third.free_energy(vt3)
    epoch_cnt += 1
    output_from_third.append(vt3.tolist())
print('\tBBRBM_Third_layer test loss : ', str(test_loss / epoch_cnt))

	BBRBM_First_layer test loss :  tensor(0.0678, grad_fn=<DivBackward0>)
	BBRBM_Second_layer test loss :  tensor(0.0223, grad_fn=<DivBackward0>)
	BBRBM_Third_layer test loss :  tensor(0.0176, grad_fn=<DivBackward0>)


In [11]:
'''Test code'''
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

output_from_first = list()
output_from_second = list()

test_loss = 0
epoch_cnt = 0

In [12]:
'''First GBRBM Guide Line'''
epoch_cnt = 0
for _, data in enumerate(torch.tensor(output_from_third)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.normal(mean=data, std=gaussian_std))

    v1, vt1, _ = rbm_first(sample_data)
    test_loss += rbm_first.free_energy(v1) - rbm_first.free_energy(vt1)
    epoch_cnt += 1
    output_from_first.append(vt1.tolist())
print('\tGBRBM_First_layer test loss : ', str(test_loss / epoch_cnt))

'''Second BBRBM Guide Line'''

for _, data in enumerate(torch.tensor(output_from_first)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.normal(mean=data, std=gaussian_std))
    
    v2, vt2, _ = rbm_second(sample_data)
    test_loss += rbm_second.free_energy(v2) - rbm_second.free_energy(vt2)
    epoch_cnt += 1
    output_from_second.append(vt2.tolist())
print('\tGBRBM_Second_layer test loss : ', str(test_loss / epoch_cnt))

'''Third BBRBM Guide Line'''

output_from_third = []
for _, data in enumerate(torch.tensor(output_from_second)):
    try:
        test_data = torch.tensor(Variable(data.clone().detach().requires_grad_(True).view(-1, BATCH_SIZE).uniform_(0, 1)), dtype=torch.float32)
    except RuntimeError:
        pass
    
    sample_data = torch.flatten(torch.bernoulli(test_data))
    
    v3, vt3, _ = rbm_third(sample_data)
    test_loss += rbm_third.free_energy(v3) - rbm_third.free_energy(vt3)
    epoch_cnt += 1
    output_from_third.append(vt3.tolist())
print('\tGBRBM_Third_layer test loss : ', str(test_loss / epoch_cnt))

	GBRBM_First_layer test loss :  tensor(0.0002, grad_fn=<DivBackward0>)
	GBRBM_Second_layer test loss :  tensor(-0.0012, grad_fn=<DivBackward0>)
	GBRBM_Third_layer test loss :  tensor(-0.0069, grad_fn=<DivBackward0>)


In [13]:
lin = nn.Linear(len(output_from_third), 4)
linear_in = torch.tensor(output_from_third)
print("Linear_in size : {}, Linear_in dim : {}".format(linear_in.size(), linear_in.dim()))

print(lin(linear_in.T))

Linear_in size : torch.Size([951, 10]), Linear_in dim : 2
tensor([[ 0.8176, -0.2903, -0.5525, -0.1088],
        [ 1.4462, -0.4073, -0.5814, -0.9102],
        [ 1.0185, -0.8875, -0.3838, -1.0311],
        [ 1.3562, -0.6270, -0.7397, -1.0249],
        [ 0.6167, -0.4450, -0.4092, -1.2821],
        [ 1.1359, -0.6909, -0.4729, -0.8698],
        [ 0.8335, -0.4097, -0.5646, -1.0012],
        [ 1.2786, -0.1553,  0.3224, -0.7126],
        [ 0.7473, -0.6915, -0.3542, -0.8980],
        [ 0.9058, -0.5644, -0.6060, -0.8304]], grad_fn=<AddmmBackward0>)
